In [1]:
import sys
import os
import pandas as pd

# Add the path to the src directory
sys.path.append(os.path.abspath(os.path.join('src')))


In [2]:
from src.postcodes import PostCodes

post_codes = PostCodes()
edinburgh_center = (55.9533, -3.1883)

post_codes.filter_by_distance(*edinburgh_center, 3)

/Users/antoine/github.com/antoinedao/nest-nomics/src/postcodes.py:16: DtypeWarning: Columns (17,57) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(full_path)


In [3]:
post_codes.df.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,LSOA21 Code,Lower layer super output area 2021,MSOA21 Code,Middle layer super output area 2021,Census output area 2021,IMD decile,Constituency Code 2024,Constituency Name 2024,Property Type,Roads
33368,EH1 1AD,Yes,55.949135,-3.192773,325610.0,673542.0,NT256735,NaN,City of Edinburgh,City Centre,...,NaN,NaN,NaN,NaN,S00143153,6.0,S14000078,Edinburgh East and Musselburgh,NaN,George Iv Bridge
33369,EH1 1AE,Yes,55.949069,-3.189408,325820.0,673532.0,NT258735,NaN,City of Edinburgh,City Centre,...,NaN,NaN,NaN,NaN,S00143121,6.0,S14000078,Edinburgh East and Musselburgh,NaN,Old Fishmarket Close
33370,EH1 1AG,Yes,55.946840,-3.188523,325871.0,673283.0,NT258732,NaN,City of Edinburgh,City Centre,...,NaN,NaN,NaN,NaN,S00143229,6.0,S14000078,Edinburgh East and Musselburgh,NaN,NaN
33371,EH1 1BB,Yes,55.951957,-3.189945,325792.0,673854.0,NT257738,NaN,City of Edinburgh,City Centre,...,NaN,NaN,NaN,NaN,S00142890,5.0,S14000078,Edinburgh East and Musselburgh,NaN,"North Bridge,Waverley Bridge"
33372,EH1 1BE,Yes,55.951961,-3.189944,325792.0,673854.0,NT257738,NaN,City of Edinburgh,City Centre,...,NaN,NaN,NaN,NaN,S00142890,5.0,S14000078,Edinburgh East and Musselburgh,NaN,NaN


In [4]:
import folium

representative_postcodes = post_codes.get_clustered_samples(40)

# Create a map centered around Edinburgh
map_center = [55.9533, -3.1883]
m = folium.Map(location=map_center, zoom_start=13)

# Add markers for each representative postcode
for postcode in representative_postcodes:
  postcode_data = post_codes.df[post_codes.df['Postcode'] == postcode].iloc[0]
  folium.Marker(
    location=[postcode_data['Latitude'], postcode_data['Longitude']],
    popup=postcode,
    icon=folium.Icon(color='blue', icon='info-sign')
  ).add_to(m)

# Display the map
m

In [ ]:
from src.rightmove.client import RightmoveAPI
import json
from datetime import datetime

def convert_datetime(obj):
	if isinstance(obj, datetime):
		return obj.isoformat()
	raise TypeError("Type not serializable")


# Fetch all properties for the list of postcodes
api = RightmoveAPI()

all_properties = await api.fetch_data_by_postcodes(representative_postcodes, sold_in=30)


df = pd.json_normalize([p.model_dump() for p in all_properties])

# Remove duplicates by address
df = df.drop_duplicates(subset='address')

df['transactions'] = df['transactions'].apply(lambda x: json.dumps(x, default=convert_datetime) if x else None)

df.to_csv('data/raw_edinburgh_properties.csv', index=False)

df.head()

Fetched page 1 out of 40 for params: {'location': 'EH6 4SQ', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for params: {'location': 'EH11 1QE', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for params: {'location': 'EH11 1BY', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for params: {'location': 'EH6 8BD', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for params: {'location': 'EH1 1DD', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for params: {'location': 'EH16 5HB', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for params: {'location': 'EH12 5TN', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for params: {'location': 'EH6 4DL', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for params: {'location': 'EH16 4LZ', 'soldIn': 30, 'radius': 0.5, 'pageNumber': 1}
Fetched page 1 out of 40 for par

,address,propertyType,bedrooms,imageInfo,hasFloorPlan,transactions,detailUrl,location.lat,location.lng,imageInfo.imageUrl,imageInfo.mediumImageUrl,imageInfo.count
0,"6, Muirhouse Green, Edinburgh, Midlothian EH4 4PQ",None,NaN,NaN,False,"[{""displayPrice"": ""\u00a3325,995"", ""dateSold"":...",https://www.rightmove.co.uk/house-prices/detai...,55.96713,-3.25228,NaN,NaN,NaN
1,"49, Flat 9, Waterfront Park, Edinburgh, Midlot...",None,NaN,NaN,False,"[{""displayPrice"": ""\u00a3157,222"", ""dateSold"":...",https://www.rightmove.co.uk/house-prices/detai...,55.97702,-3.25116,NaN,NaN,NaN
2,"10 Flat 2, Mottram Road, Edinburgh, Midlothian...",None,NaN,NaN,False,"[{""displayPrice"": ""\u00a3199,000"", ""dateSold"":...",https://www.rightmove.co.uk/house-prices/detai...,55.96911,-3.25171,NaN,NaN,NaN
3,"6a, Muirhouse Green, Edinburgh, Midlothian EH4...",None,NaN,NaN,False,"[{""displayPrice"": ""\u00a3325,995"", ""dateSold"":...",https://www.rightmove.co.uk/house-prices/detai...,55.96713,-3.25228,NaN,NaN,NaN
4,"757/4, Ferry Road, Edinburgh, Midlothian EH4 2UB",None,NaN,NaN,False,"[{""displayPrice"": ""\u00a3135,000"", ""dateSold"":...",https://www.rightmove.co.uk/house-prices/detai...,55.96592,-3.25625,NaN,NaN,NaN


In [11]:
from datetime import datetime

def convert_datetime(obj):
	if isinstance(obj, datetime):
		return obj.isoformat()
	raise TypeError("Type not serializable")

df['transactions'] = df['transactions'].apply(lambda x: json.dumps(x, default=convert_datetime) if x else None)

df.to_csv('data/raw_edinburgh_properties.csv', index=False)
